# Ablation: Semi-Supervised Architope (Financial Data)
---
- This code Implements Algorithm 3.2 of the "PC-NNs" paper.

#### Mode: Code-Testin Parameter(s)

In [1]:
trial_run = True

### Meta-parameters

In [2]:
# Test-size Ratio
test_size_ratio = 0.1
min_height = 50
# Ablation Finess
N_plot_finess = 15
# min_parts_threshold = .001; max_parts_threshold = 0.9
N_min_parts = 2; N_max_plots = 10
Tied_Neurons_Q = False

In [3]:
#------------------------------------#
# Only For Motivational Example Only #
#------------------------------------#
## Hyperparameters
percentage_in_row = .25
N = 10**4

def f_1(x):
    return x
def f_2(x):
    return x**2
x_0 = 0
x_end = 1
import seaborn as sns

### Hyperparameters

Only turn of if running code directly here, typically this script should be run be called by other notebooks.  

In [4]:
# load dataset
results_path = "./outputs/models/"
results_tables_path = "./outputs/results/"
raw_data_path_folder = "./inputs/raw/"
data_path_folder = "./inputs/data/"

### Import

In [5]:
# Load Packages/Modules
exec(open('Init_Dump.py').read())
# Load Hyper-parameter Grid
exec(open('Grid_Enhanced_Network.py').read())
# Load Helper Function(s)
exec(open('Helper_Functions.py').read())
# Pre-process Data
if Option_Function != "Motivational_Example": 
    exec(open('Financial_Data_Preprocessor.py').read())
else:
    print(1)
    exec(open('Motivational_Example.py').read())
    print("Training Data size: ",X_train.shape[0])
# Import time separately
import time

# TEMP
# import pickle_compat
# pickle_compat.patch()
# param_grid_Vanilla_Nets['input_dim']=X_train.shape[1]

Using TensorFlow backend.


Deep Feature Builder - Ready
Deep Classifier - Ready
1
Training Data size:  9000


### Set Seed

In [6]:
random.seed(2021)
tf.random.set_seed(2021)

#### Pre-Process:
- Convert Categorical Variables to Dummies
- Remove Bad Column
- Perform Training/Test Split

# Random Lipschitz Partition Builder

We implement the random paritioning method of [Yair Bartal](https://scholar.google.com/citations?user=eCXP24kAAAAJ&hl=en):
- [On approximating arbitrary metrices by tree metrics](https://dl.acm.org/doi/10.1145/276698.276725)

The algorithm is summarized as follow:

---

## Algorithm:
 1. Sample $\alpha \in [4^{-1},2^{-1}]$ randomly and uniformly,
 2. Apply a random suffle of the data, (a random bijection $\pi:\{i\}_{i=1}^X \rightarrow \mathbb{X}$),
 3. For $i = 1,\dots,I$:
   - Set $K_i\triangleq B\left(\pi(i),\alpha \Delta \right) - \bigcup_{j=1}^{i-1} P_j$
 
 4. Remove empty members of $\left\{K_i\right\}_{i=1}^X$.  
 
 **Return**: $\left\{K_i\right\}_{i=1}^{\tilde{X}}$.  
 
 For more details on the random-Lipschitz partition of Yair Bartal, see this [well-written blog post](https://nickhar.wordpress.com/2012/03/26/lecture-22-random-partitions-of-metric-spaces/).

## Define Random Partition Builder

#### Explicit Partion Builder:
Implements exactly Algorithm 2:

In [7]:
def Random_Lipschitz_Partioner(X_in,
                               y_in,
                               N_parts_to_get=4):

    # Compute Size of each part
    size_part_reference = int(round(X_in.shape[0]/N_parts_to_get))

    # Apply random bijection #
    #------------------------#
    ## Get random bijection indices
    random_bijection_indices = np.random.choice(range(X_in.shape[0]),size=X_in.shape[0], replace=False)
    ## Apply random bijections
    X_in_shuffled = X_in[random_bijection_indices,:]
    y_in_shuffled = y_in[random_bijection_indices,:]

    # Initialize Lists #
    #------------------#
    X_parts = []
    y_parts = []

    for i_th_part_to_get in range(N_parts_to_get):
        # Build random balls #
        #--------------------#
        ## Sample random radius
        size_part = int(np.maximum(1,np.round(size_part_reference*np.random.uniform(low=.5,high=1.5,size=1)[0])))
        ## Sample random point
        X_center_loop_index = np.random.choice(range(X_in_shuffled.shape[0]),size=1, replace=False)
        X_center_loop = X_in_shuffled[X_center_loop_index,:]
        ## Compute Typical Distances from Center
        distances_loop = X_center_loop-X_in_shuffled
        distances_loop = np.linalg.norm(distances_loop, axis=1)

        # Remove Random Ball from Dataset
        if size_part <= len(distances_loop):
            ## Identify indices
            indices_smallest_to_random_ball = np.argsort(distances_loop)[:size_part]
        else:
            print('Final Loop')
            indices_smallest_to_random_ball = np.array(range(X_in_shuffled.shape[0]))
        ## Extract Parts
        X_current_part_loop = X_in_shuffled[indices_smallest_to_random_ball,:]
        y_current_part_loop = y_in_shuffled[indices_smallest_to_random_ball,:]
        ## Append to List of Parts
        X_parts.append(X_current_part_loop)
        y_parts.append(y_current_part_loop)

        # Remove Selected Entries From Array #
        #------------------------------------#
        X_in_shuffled = np.delete(X_in_shuffled,indices_smallest_to_random_ball,axis=0)
        y_in_shuffled = np.delete(y_in_shuffled,indices_smallest_to_random_ball,axis=0)

        # Failsafe if procedure has terminated
        if X_in_shuffled.shape[0] == 0:
            print('breaking early')
            break
    # Count Number of Parts Generated        
    N_parts_generated = len(X_parts)
    # Output Parts
    return X_parts, y_parts, N_parts_generated

# Ablation Function

In [8]:
def get_PCNNs(N_parts,X_train,y_train,X_test,y_test):

    # Initialization(s) #
    #-------------------#
    N_neurons = 0
    L_timer = 0
    P_timer = 0
    Mean_Width_Subnetworks = 0

    # Partitioner Begin #
    #-------------------#
    import time
    partitioning_time_begin = time.time()
    print('-------------------------------------------------------')
    print('Randomly Initialized Parts - Via Randomized Algorithm 2')
    print('-------------------------------------------------------')
    X_parts_list, y_parts_list, N_parts_Generated_by_Algo_2 = Random_Lipschitz_Partioner(X_train.to_numpy(),
                                                                                         y_train.reshape(-1,1),
                                                                                         N_parts)
    partitioning_time = time.time() - partitioning_time_begin
    print('The_parts_listhe number of parts are: ' + str(N_parts_Generated_by_Algo_2)+'.')
    ############# Partitioner End ########

    print('-----------------------------------------------------')
    print('Training Sub-Networks on Each Randomly Generated Part')
    print('-----------------------------------------------------')
    # Time-Elapse (Start) for Training on Each Part #
    PCNN_timer = time.time(); PCNN_timer = -math.inf; N_params_Architope = 0; N_params_tally = 0
    # Remove Eager Execution Error(s)
    tf.compat.v1.disable_eager_execution()
    # Automatically Initialize Correct Input/Output Dimension(s)
    param_grid_Vanilla_Nets['input_dim'] = [X_train.shape[1]]; param_grid_Vanilla_Nets['output_dim'] = [1]
    param_grid_Deep_Classifier['input_dim'] = [X_train.shape[1]]
    # Decide if/or not to tie neuron numbers of sub-patterns together
    if Tied_Neurons_Q == True:
        param_grid_Vanilla_Nets['height'] = [int(np.maximum(round(param_grid_Vanilla_Nets['height'][0]/N_parts),min_height))]
        param_grid_Deep_Classifier['height'] = [int(np.maximum(round(param_grid_Deep_Classifier['height'][0]/N_parts),min_height))]

    for current_part in range(N_parts_Generated_by_Algo_2):
        # Update User #
        #-------------#
        print('-----------------------------------------------------------')
        print('Currently Training Part: '+str(current_part)+'/'+str(N_parts_Generated_by_Algo_2 )+'Total Parts.')
        print('-----------------------------------------------------------')

        # Timer for Part
        part_training_timer = time.time()
        # Get Data for Sub-Pattern
        X_loop = pd.DataFrame(X_parts_list[current_part])
        y_loop = (y_parts_list[current_part]).reshape(-1,)
        # Train ffNN
        y_hat_part_loop, y_hat_part_loop_test, N_neurons_PCNN_loop = build_ffNN(n_folds = 4, 
                                                                                          n_jobs = n_jobs,
                                                                                          n_iter = n_iter, 
                                                                                          param_grid_in = param_grid_Vanilla_Nets, 
                                                                                          X_train= X_loop, 
                                                                                          y_train=y_loop,
                                                                                          X_test_partial=X_train,
                                                                                          X_test=X_test,
                                                                                          NOCV=True)
        # Reshape y
        ## Training
        y_train.shape = (y_train.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        y_hat_part_loop.shape = (y_hat_part_loop.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        ## Testing
        y_test.shape = (y_test.shape[0], param_grid_Vanilla_Nets['output_dim'][0])
        y_hat_part_loop_test.shape = (y_hat_part_loop_test.shape[0], param_grid_Vanilla_Nets['output_dim'][0])

        # Append predictions to data-frames
        ## If first prediction we initialize data-frames
        if current_part==0:
            # Register quality
            training_quality = np.array(np.abs(y_hat_part_loop-y_train)).reshape(y_hat_part_loop.shape[0],1)

            # Save Predictions
            predictions_train = y_hat_part_loop.reshape(y_hat_part_loop.shape[0],1)
            predictions_test = y_hat_part_loop_test.reshape(y_hat_part_loop_test.shape[0],1)


        ## If not first prediction we append to already initialized dataframes
        else:
        # Register Best Scores
            #----------------------#
            # Write Predictions 
            # Save Predictions
            y_hat_train_loop = y_hat_part_loop.reshape(predictions_train.shape[0],1)
            predictions_train = np.append(predictions_train,y_hat_train_loop,axis=1)
            y_hat_test_loop = y_hat_part_loop_test.reshape(predictions_test.shape[0],1)
            predictions_test = np.append(predictions_test,y_hat_test_loop,axis=1)

            # Evaluate Errors #
            #-----------------#
            # Training
            prediction_errors = np.abs(y_hat_train_loop-y_train)
            training_quality = np.append(training_quality,prediction_errors.reshape(training_quality.shape[0],1),axis=1)

        #==============================#
        # Update Performance Metric(s) #
        #==============================#
        part_training_timer = time.time() - part_training_timer
        # L-Time
        L_timer += partitioning_time
        # P-Time
        P_timer = max(P_timer,part_training_timer)
        # N. Params
        N_neurons += N_neurons_PCNN_loop
        # Mean Width for Sub-Network(s)
        Mean_Width_Subnetworks += param_grid_Vanilla_Nets['height'][0]

    # Take Mean of Width(s)
    Mean_Width_Subnetworks = Mean_Width_Subnetworks/N_parts_Generated_by_Algo_2
    print('-----------------------')
    print('Training Deep Zero-Sets')
    print('-----------------------')


    # Time Elapsed for Training Deep Zero-Sets
    Deep_Zero_Sets_timer = time.time()

    ## Initialize Classes Labels
    partition_labels_training_integers = np.argmin(training_quality,axis=-1)
    partition_labels_training = pd.DataFrame(pd.DataFrame(partition_labels_training_integers) == 0)
    ## Build Classes
    for part_column_i in range(1,(training_quality.shape[1])):
        partition_labels_training = pd.concat([partition_labels_training,
                                               (pd.DataFrame(partition_labels_training_integers) == part_column_i)
                                              ],axis=1)
    ## Convert to integers
    partition_labels_training = partition_labels_training+0
    ## Train simple deep classifier
    tf.compat.v1.disable_eager_execution()
    predicted_classes_train, predicted_classes_test, N_params_deep_classifier = build_simple_deep_classifier(n_folds = CV_folds, 
                                                                                                        n_jobs = n_jobs, 
                                                                                                        n_iter =n_iter, 
                                                                                                        param_grid_in=param_grid_Deep_Classifier, 
                                                                                                        X_train = X_train.values, 
                                                                                                        y_train = partition_labels_training.values,
                                                                                                        X_test = X_test.values)
    # Get Binary Classes (Discontinuous Unit)
    ## Training Set
    predicted_classes_train = ((predicted_classes_train>.5)*1).astype(int)
    ## Testing Set
    predicted_classes_test = ((predicted_classes_test > .5)*1).astype(int)
    # Get PC-NN Prediction(s)
    ## Train
    PCNN_prediction_y_train = (predictions_train*predicted_classes_train).sum(axis=1)
    ## Test
    PCNN_prediction_y_test = (predictions_test*predicted_classes_test).sum(axis=1)

    # End Timer
    Deep_Zero_Sets_timer = time.time() - Deep_Zero_Sets_timer

    print('-----------------------------------')
    print('Computing Final Performance Metrics')
    print('-----------------------------------')
    # Time-Elapsed Training Deep Classifier

    # Update Times
    L_timer +=Deep_Zero_Sets_timer
    P_timer +=Deep_Zero_Sets_timer
    # Update Number of Neurons Used
    N_neurons += (param_grid_Deep_Classifier['height'][0])*(param_grid_Deep_Classifier['depth'][0])



    # Compute Peformance
    performance_PCNN = reporter(y_train_hat_in=PCNN_prediction_y_train,y_test_hat_in=PCNN_prediction_y_test,
                                y_train_in=y_train,
                                y_test_in=y_test)
    # Write Performance
    performance_PCNN.to_latex((results_tables_path+"PCNN_full_performance.tex"))

    # Update User
    print(performance_PCNN)

    ### Model Complexity/Efficiency Metrics
    # Build AIC-like Metric #
    #-----------------------#
    AIC_like = 2*(N_neurons - np.log((performance_PCNN['test']['MAE'])))
    AIC_like = np.round(AIC_like,3)
    Efficiency = np.log(N_neurons) *(performance_PCNN['test']['MAE'])
    Efficiency = np.round(Efficiency,3)


    # Build Table #
    #-------------#
    PCNN_Model_Complexity = pd.DataFrame({'L-time': [L_timer],
                                               'P-time':[P_timer],
                                               'N_params_expt': [N_neurons],
                                               'AIC-like': [AIC_like],
                                               'Eff': [Efficiency],
                                               'N. Parts':[N_parts_Generated_by_Algo_2]})


    # Write Required Training Time(s)
    PCNN_Model_Complexity.to_latex((results_tables_path+"PCNN_full_model_complexities.tex"))

    #--------------======---------------#
    # Display Required Training Time(s) #
    #--------------======---------------#
    print(PCNN_Model_Complexity)
    
    
    # Return Output(s)
    return performance_PCNN, PCNN_Model_Complexity, N_parts_Generated_by_Algo_2, N_neurons, Mean_Width_Subnetworks

## Ablation Function
*Made for 1-D output...for multidimensional y just remove reshaping in code.*

---
# Perform Ablation:
---

In [ ]:
# Initialize 
# q_implicit_N_parts_possibilities = np.linspace(min_parts_threshold,max_parts_threshold,N_plot_finess)
N_parts_possibilities = np.unique(np.round(np.linspace(N_min_parts,N_max_plots,num=N_plot_finess))).astype(int)

# Get Performance Metric
for inplicit_N_parts_loop in range(len(N_parts_possibilities)):
    ### UPDATE USER ###
    for k in range(15):
        print('--------------------------------------')
    print('Ablation Completion Percentage:',(inplicit_N_parts_loop/N_plot_finess))
    for k in range(15):
        print('--------------------------------------')
    
    # Implicitly Set: Current Number of Parts
#     q_implicit_N_parts_loop = q_implicit_N_parts_possibilities[inplicit_N_parts_loop]
    N_parts_possibilities_loop = N_parts_possibilities[inplicit_N_parts_loop]
    # Run Algos. 1+2
    performance_Architope_loop, Architope_Model_Complexity_full_loop, N_parts_Generated_by_Algo_2_loop, N_params_architope_loop, height_mean_loop = get_PCNNs(N_parts_possibilities_loop,X_train,y_train,X_test,y_test)
#     performance_Architope_loop, Architope_Model_Complexity_full_loop, N_parts_Generated_by_Algo_2_loop, N_params_architope_loop, height_mean_loop = Ablate_PCNNs(q_implicit_N_parts_loop,data_y,X_train,X_test,y_test)
    # Reshape
    performance_Architope_loop = performance_Architope_loop.to_numpy().reshape([3,2,1])
    Architope_Model_Complexity_full_loop = Architope_Model_Complexity_full_loop.to_numpy().reshape([1,6,1])

    # Record
    if inplicit_N_parts_loop == 0:
        performance_Architope_history = performance_Architope_loop
        Architope_Model_Complexity_history = Architope_Model_Complexity_full_loop
        N_parts_Generated_by_Algo_2_history = N_parts_Generated_by_Algo_2_loop
        N_params_architope_hist = N_params_architope_loop
        height_mean_hist = height_mean_loop
    else:
        performance_Architope_history = np.concatenate((performance_Architope_history,performance_Architope_loop),axis=2)
        Architope_Model_Complexity_history = np.concatenate((Architope_Model_Complexity_history,Architope_Model_Complexity_full_loop),axis=2)
        N_parts_Generated_by_Algo_2_history = np.append(N_parts_Generated_by_Algo_2_history,N_parts_Generated_by_Algo_2_loop)
        N_params_architope_hist = np.append(N_params_architope_hist,N_params_architope_loop)
        height_mean_hist = np.append(height_mean_hist,height_mean_loop)

--------------------------------------
--------------------------------------
--------------------------------------
Ablation Completion Percentage: 0.0
--------------------------------------
--------------------------------------
--------------------------------------
-------------------------------------------------------
Randomly Initialized Parts - Via Randomized Algorithm 2
-------------------------------------------------------
The_parts_listhe number of parts are: 2.
-----------------------------------------------------
Training Sub-Networks on Each Randomly Generated Part
-----------------------------------------------------
-----------------------------------------------------------
Currently Training Part: 0/2Total Parts.
-----------------------------------------------------------
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 2675 samples
Epoch 1/100
2675/2675 [==============================] - 0s 151us/sample - loss: 0.1866 - mse: 

2675/2675 [==============================] - 0s 82us/sample - loss: 0.0617 - mse: 0.0122 - mae: 0.0617 - mape: 21.5028
Epoch 54/100
2675/2675 [==============================] - 0s 80us/sample - loss: 0.0618 - mse: 0.0121 - mae: 0.0618 - mape: 21.5760
Epoch 55/100
2675/2675 [==============================] - 0s 75us/sample - loss: 0.0617 - mse: 0.0122 - mae: 0.0617 - mape: 21.4702
Epoch 56/100
2675/2675 [==============================] - 0s 87us/sample - loss: 0.0616 - mse: 0.0121 - mae: 0.0616 - mape: 21.4518
Epoch 57/100
2675/2675 [==============================] - 0s 84us/sample - loss: 0.0618 - mse: 0.0122 - mae: 0.0618 - mape: 21.5779
Epoch 58/100
2675/2675 [==============================] - 0s 80us/sample - loss: 0.0617 - mse: 0.0122 - mae: 0.0617 - mape: 21.4327
Epoch 59/100
2675/2675 [==============================] - 0s 79us/sample - loss: 0.0617 - mse: 0.0121 - mae: 0.0617 - mape: 21.5898
Epoch 60/100
2675/2675 [==============================] - 0s 81us/sample - loss: 0.0617 -

6304/6304 [==============================] - 0s 70us/sample - loss: 0.0740 - mse: 0.0073 - mae: 0.0740 - mape: 151.2223
Epoch 14/100
6304/6304 [==============================] - 0s 72us/sample - loss: 0.0739 - mse: 0.0073 - mae: 0.0739 - mape: 151.1891
Epoch 15/100
6304/6304 [==============================] - 0s 76us/sample - loss: 0.0739 - mse: 0.0073 - mae: 0.0739 - mape: 153.0473
Epoch 16/100
6304/6304 [==============================] - 0s 78us/sample - loss: 0.0739 - mse: 0.0073 - mae: 0.0739 - mape: 151.5273
Epoch 17/100
6304/6304 [==============================] - 1s 103us/sample - loss: 0.0739 - mse: 0.0073 - mae: 0.0739 - mape: 152.3679
Epoch 18/100
6304/6304 [==============================] - 1s 100us/sample - loss: 0.0738 - mse: 0.0073 - mae: 0.0738 - mape: 152.4537
Epoch 19/100
6304/6304 [==============================] - 1s 114us/sample - loss: 0.0737 - mse: 0.0073 - mae: 0.0737 - mape: 150.6694
Epoch 20/100
6304/6304 [==============================] - 1s 90us/sample - loss

6304/6304 [==============================] - 0s 72us/sample - loss: 0.0717 - mse: 0.0069 - mae: 0.0717 - mape: 147.2756
Epoch 75/100
6304/6304 [==============================] - 0s 73us/sample - loss: 0.0718 - mse: 0.0069 - mae: 0.0718 - mape: 143.1993
Epoch 76/100
6304/6304 [==============================] - 0s 70us/sample - loss: 0.0718 - mse: 0.0069 - mae: 0.0718 - mape: 145.5057
Epoch 77/100
6304/6304 [==============================] - 0s 71us/sample - loss: 0.0718 - mse: 0.0069 - mae: 0.0718 - mape: 144.0064
Epoch 78/100
6304/6304 [==============================] - 0s 71us/sample - loss: 0.0716 - mse: 0.0069 - mae: 0.0716 - mape: 146.4583
Epoch 79/100
6304/6304 [==============================] - 0s 72us/sample - loss: 0.0716 - mse: 0.0069 - mae: 0.0716 - mape: 144.8864
Epoch 80/100
6304/6304 [==============================] - 0s 71us/sample - loss: 0.0716 - mse: 0.0069 - mae: 0.0716 - mape: 147.0248
Epoch 81/100
6304/6304 [==============================] - 0s 73us/sample - loss: 0

/Users/kratsi0000/opt/anaconda3/envs/bpcnn/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide
/Users/kratsi0000/opt/anaconda3/envs/bpcnn/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in true_divide


...

2440/2440 [==============================] - 0s 80us/sample - loss: 0.0518 - mse: 0.0080 - mae: 0.0518 - mape: 7.7330
Epoch 24/100
2440/2440 [==============================] - 0s 91us/sample - loss: 0.0518 - mse: 0.0080 - mae: 0.0518 - mape: 7.7930
Epoch 25/100
2440/2440 [==============================] - 0s 82us/sample - loss: 0.0517 - mse: 0.0080 - mae: 0.0517 - mape: 7.7511
Epoch 26/100
2440/2440 [==============================] - 0s 84us/sample - loss: 0.0518 - mse: 0.0080 - mae: 0.0518 - mape: 7.7922
Epoch 27/100
2440/2440 [==============================] - 0s 84us/sample - loss: 0.0520 - mse: 0.0079 - mae: 0.0520 - mape: 7.9432
Epoch 28/100
2440/2440 [==============================] - 0s 80us/sample - loss: 0.0518 - mse: 0.0079 - mae: 0.0518 - mape: 7.8488
Epoch 29/100
2440/2440 [==============================] - 0s 83us/sample - loss: 0.0517 - mse: 0.0079 - mae: 0.0517 - mape: 7.7935
Epoch 30/100
2440/2440 [==============================] - 0s 83us/sample - loss: 0.0518 - mse: 0

2440/2440 [==============================] - 0s 89us/sample - loss: 0.0518 - mse: 0.0078 - mae: 0.0518 - mape: 7.9193
Epoch 86/100
2440/2440 [==============================] - 0s 80us/sample - loss: 0.0517 - mse: 0.0078 - mae: 0.0517 - mape: 7.8459
Epoch 87/100
2440/2440 [==============================] - 0s 83us/sample - loss: 0.0516 - mse: 0.0079 - mae: 0.0516 - mape: 7.7819
Epoch 88/100
2440/2440 [==============================] - 0s 89us/sample - loss: 0.0516 - mse: 0.0080 - mae: 0.0516 - mape: 7.7517
Epoch 89/100
2440/2440 [==============================] - 0s 82us/sample - loss: 0.0516 - mse: 0.0080 - mae: 0.0516 - mape: 7.7624
Epoch 90/100
2440/2440 [==============================] - 0s 87us/sample - loss: 0.0517 - mse: 0.0080 - mae: 0.0517 - mape: 7.7572
Epoch 91/100
2440/2440 [==============================] - 0s 82us/sample - loss: 0.0516 - mse: 0.0079 - mae: 0.0516 - mape: 7.7588
Epoch 92/100
2440/2440 [==============================] - 0s 91us/sample - loss: 0.0518 - mse: 0

Epoch 46/100
1816/1816 [==============================] - 0s 92us/sample - loss: 0.0657 - mse: 0.0083 - mae: 0.0657 - mape: 39.3855
Epoch 47/100
1816/1816 [==============================] - 0s 93us/sample - loss: 0.0657 - mse: 0.0083 - mae: 0.0657 - mape: 39.1693
Epoch 48/100
1816/1816 [==============================] - 0s 92us/sample - loss: 0.0657 - mse: 0.0085 - mae: 0.0657 - mape: 38.5027
Epoch 49/100
1816/1816 [==============================] - 0s 90us/sample - loss: 0.0656 - mse: 0.0085 - mae: 0.0656 - mape: 38.2725
Epoch 50/100
1816/1816 [==============================] - 0s 90us/sample - loss: 0.0656 - mse: 0.0085 - mae: 0.0656 - mape: 38.2222
Epoch 51/100
1816/1816 [==============================] - 0s 87us/sample - loss: 0.0656 - mse: 0.0085 - mae: 0.0656 - mape: 38.0740
Epoch 52/100
1816/1816 [==============================] - 0s 89us/sample - loss: 0.0656 - mse: 0.0086 - mae: 0.0656 - mape: 37.9932
Epoch 53/100
1816/1816 [==============================] - 0s 90us/sample - l

2449/2449 [==============================] - 0s 76us/sample - loss: 0.0864 - mse: 0.0265 - mae: 0.0864 - mape: 126.4903
Epoch 7/100
2449/2449 [==============================] - 0s 83us/sample - loss: 0.0759 - mse: 0.0164 - mae: 0.0759 - mape: 159.3596
Epoch 8/100
2449/2449 [==============================] - 0s 80us/sample - loss: 0.0634 - mse: 0.0081 - mae: 0.0634 - mape: 201.1561
Epoch 9/100
2449/2449 [==============================] - 0s 77us/sample - loss: 0.0492 - mse: 0.0032 - mae: 0.0492 - mape: 243.9325
Epoch 10/100
2449/2449 [==============================] - 0s 77us/sample - loss: 0.0419 - mse: 0.0028 - mae: 0.0419 - mape: 300.1676
Epoch 11/100
2449/2449 [==============================] - 0s 77us/sample - loss: 0.0419 - mse: 0.0028 - mae: 0.0419 - mape: 309.4127
Epoch 12/100
2449/2449 [==============================] - 0s 76us/sample - loss: 0.0419 - mse: 0.0027 - mae: 0.0419 - mape: 296.5345
Epoch 13/100
2449/2449 [==============================] - 0s 81us/sample - loss: 0.04

2449/2449 [==============================] - 0s 82us/sample - loss: 0.0410 - mse: 0.0027 - mae: 0.0410 - mape: 298.1930
Epoch 68/100
2449/2449 [==============================] - 0s 98us/sample - loss: 0.0409 - mse: 0.0027 - mae: 0.0409 - mape: 298.6344
Epoch 69/100
2449/2449 [==============================] - 0s 94us/sample - loss: 0.0409 - mse: 0.0027 - mae: 0.0409 - mape: 297.6317
Epoch 70/100
2449/2449 [==============================] - 0s 91us/sample - loss: 0.0409 - mse: 0.0027 - mae: 0.0409 - mape: 293.2399
Epoch 71/100
2449/2449 [==============================] - 0s 84us/sample - loss: 0.0409 - mse: 0.0027 - mae: 0.0409 - mape: 299.7372
Epoch 72/100
2449/2449 [==============================] - 0s 76us/sample - loss: 0.0409 - mse: 0.0026 - mae: 0.0409 - mape: 295.2034
Epoch 73/100
2449/2449 [==============================] - 0s 78us/sample - loss: 0.0408 - mse: 0.0027 - mae: 0.0408 - mape: 292.7605
Epoch 74/100
2449/2449 [==============================] - 0s 76us/sample - loss: 0

In [ ]:
# Randomization may produce duplicates; we remove these with the following snippet:
get_unique_entries = np.unique(N_parts_Generated_by_Algo_2_history, return_index=True)[1]
N_parts_Generated_by_Algo_2_history_report = N_parts_Generated_by_Algo_2_history[get_unique_entries]
## Prediction Qualities
performance_Architope_history_report_MAE_train = (performance_Architope_history[1,0,:])[get_unique_entries]
performance_Architope_history_report_MAE_test = (performance_Architope_history[1,1,:])[get_unique_entries]
performance_Architope_history_report_MSE_train = (performance_Architope_history[0,0,:])[get_unique_entries]
performance_Architope_history_report_MSE_test = (performance_Architope_history[0,1,:])[get_unique_entries]
## Model Complexities
L_Times = (Architope_Model_Complexity_history[:,0].reshape(-1,))[get_unique_entries]
P_Times = (Architope_Model_Complexity_history[:,1].reshape(-1,))[get_unique_entries]
N_Params = (N_params_architope_hist.reshape(-1,))[get_unique_entries]
height_mean_hist = (height_mean_hist.reshape(-1,))[get_unique_entries]
AIC_Like = (Architope_Model_Complexity_history[:,3].reshape(-1,))[get_unique_entries]
Eff = (Architope_Model_Complexity_history[:,4].reshape(-1,))[get_unique_entries]

---
## Plots
---

## MAE

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("MAE")
plt.xlabel("N. Parts")
plt.ylabel("MAE")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_Architope_history_report_MAE_test)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_MAE_test___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

## MSE

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("MSE")
plt.xlabel("N. Parts")
plt.ylabel("MSE")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         performance_Architope_history_report_MSE_test)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_MSE___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

# L-Time

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("L-Time")
plt.xlabel("N. Parts")
plt.ylabel("L-Time")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         L_Times)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_L_Time___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

# P-Times

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("P-Time")
plt.xlabel("N. Parts")
plt.ylabel("P-Time")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         P_Times)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_P_Time___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

## N-Params

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("N. Params")
plt.xlabel("N. Parts")
plt.ylabel("N. Params")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         N_Params)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_N_Params___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

# Mean Widths for Sub-Pattern Networks

In [ ]:
sns.set()
# Initialize Plot #
#-----------------#
plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# Format Plot #
#-------------#
plt.title("Mean SubNetwork Widths")
plt.xlabel("N. Parts")
plt.ylabel("Mean SubNetwork Widths")

# Generate Plots #
#----------------#
# Plot Signal
plt.plot(N_parts_Generated_by_Algo_2_history_report,
         height_mean_hist)


# Export #
#--------#
# SAVE Figure to .eps
plt.savefig('./outputs/plotsANDfigures/Ablation_Mean_Widths___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
plt.show()

## AIC-Like

In [ ]:
# sns.set()
# # Initialize Plot #
# #-----------------#
# plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# # Format Plot #
# #-------------#
# plt.title("AIC-Like")
# plt.xlabel("N. Parts")
# plt.ylabel("AIC-Like")

# # Generate Plots #
# #----------------#
# # Plot Signal
# plt.plot(N_parts_Generated_by_Algo_2_history_report,
#          AIC_Like)


# # Export #
# #--------#
# # SAVE Figure to .eps
# plt.savefig('./outputs/plotsANDfigures/Ablation_AIC_Like___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
# plt.show()

## Efficiency Metric

In [ ]:
# sns.set()
# # Initialize Plot #
# #-----------------#
# plt.figure(num=None, figsize=(12, 12), dpi=80, facecolor='w', edgecolor='k')
# # Format Plot #
# #-------------#
# plt.title("Eff")
# plt.xlabel("N. Parts")
# plt.ylabel("Eff")

# # Generate Plots #
# #----------------#
# # Plot Signal
# plt.plot(N_parts_Generated_by_Algo_2_history_report,
#          Eff)


# # Export #
# #--------#
# # SAVE Figure to .eps
# plt.savefig('./outputs/plotsANDfigures/Ablation_Eff___'+str(Option_Function)+'__Fix_Neurons_Q'+str(Tied_Neurons_Q)+'.eps')
# plt.show()

---
# Fin
---

---

---